In [84]:
import pandas as pd
import requests
import numpy as np
from sklearn.metrics import root_mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import root_mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split

In [58]:
player_id = 355 
url_player = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
api_result_player = requests.get(url_player).json()
stats = api_result_player['history']
df_stats = pd.json_normalize(stats)
df_stats.index = np.arange(1, len(df_stats) + 1)
features = ['opponent_team', 'was_home', 'minutes', 'points', 'goals', 'assists', 'expected_goals', 'expected_assists']
df_stats.columns.values

features = ['player_id', 'opponent_team', 'was_home', 'total_points']

In [59]:
df_stats = df_stats[features]
df_stats['avg_points'] = df_stats.total_points.rolling(window=3, closed = 'left', min_periods = 1).mean()

In [60]:
df_stats.avg_points = df_stats.avg_points.fillna(1)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(df_stats[['avg_points']], df_stats['total_points'], test_size=0.3, random_state=0)

In [79]:
class MyBaselineEstimator(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.column_name]

    def predict(self, X):
        return self.transform(X)

In [85]:
from sklearn.pipeline import Pipeline

pipeline_champion = Pipeline([
    ('classifier', MyBaselineEstimator('avg_points')),
])
pipeline_challenger = Pipeline([
    ('LinearRegression', LinearRegression())
     ])

In [93]:
def compare_two_pipelines_performance(pipeline_champion, pipeline_challenger,
                                      X_train, X_test, y_train, y_test):
    # Train pipeline1
    pipeline_champion.fit(X_train, y_train)
    y_pred_champion = pipeline_champion.predict(X_test)
    # Train pipeline2
    pipeline_challenger.fit(X_train, y_train)
    y_pred_challenger = pipeline_challenger.predict(X_test)
    # Compute the accuracy of pipeline1
    champion_accuracy = root_mean_squared_error(y_test, y_pred_champion)
    # Compute the accuracy of pipeline2
    challenger_accuracy = root_mean_squared_error(y_test, y_pred_challenger)
    print(f"pipeline_champion accuracy: {champion_accuracy}")
    print(f"pipeline_challenger accuracy: {challenger_accuracy}")
    return champion_accuracy, challenger_accuracy

In [94]:
compare_two_pipelines_performance(pipeline_champion, pipeline_challenger, X_train, X_test, y_train, y_test)

pipeline_champion accuracy: 4.288798299019575
pipeline_challenger accuracy: 4.351701086384168


(4.288798299019575, 4.351701086384168)

In [98]:
###  TO DO: DODAĆ DO df_players_stats KOLUMNĘ avg_points A NASTĘPNIE WYKONAĆ PORÓWNANIE PIPELINE